In [5]:
import selenium
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time # dėl sleep komandos

opcijos = Options()
opcijos.add_argument('--incognito')

driver = uc.Chrome(options=opcijos)

gamintojai = []
modeliai = []
metai = []
kebulo_tipas = []
kuro_rusis = []
pavaros = []
turiai = []
galios = []
ridos = []
miestai = []
kainos = []

for i in range(1,201):
    url = "https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr="+str(i)
    driver.get(url)
    if i in [1, 40, 80, 120, 160]: time.sleep(15)
    else: time.sleep(1)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    
    ResultsSet = bs.find('div', {'class':'auto-lists lt'}).find_all('div', {'class':'announcement-body'})

    for skelbimas in ResultsSet:
            announcement_title = skelbimas.find('div', {'class':'announcement-title'}).text.strip()
            gamintojai.append(announcement_title.split()[0])
            modeliai.append(announcement_title.split(maxsplit=1)[1])          
            metai.append(int(skelbimas.find('div', {'class':'announcement-title-parameters'}).find('div', {'class':'announcement-parameters'}).contents[1].text.strip()[:4]))
            kebulo_tipas.append(skelbimas.find('div', {'class':'announcement-title-parameters'}).find('div', {'class':'announcement-parameters'}).contents[3].text.strip())
            kainos.append(skelbimas.find('div', {'class':'announcement-pricing-info'}).contents[1].text.strip().replace(' ','')[:-1])

            atributu_kiekis = len(skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents)

            if atributu_kiekis == 7:
                kuras = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[1].text.strip()
                pavara = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[3].text.strip()
                variklis = 'Nenurodyta'
                turis = 'Nenurodyta'
                galia = 'Nenurodyta'
                rida = 'Nenurodyta'
                miestas = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[5].text.strip()
            elif atributu_kiekis == 9:
                kuras = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[1].text.strip()
                pavara = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[3].text.strip()
                variklis = 'Nenurodyta'
                turis = 'Nenurodyta'
                galia = 'Nenurodyta'
                rida = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[5].text.strip().replace(' ','')[:-2]
                miestas = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[7].text.strip()
            elif atributu_kiekis == 11:
                kuras = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[1].text.strip()
                pavara = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[3].text.strip()
                variklis = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[5].text.strip()
                turis = float(variklis.split()[0])
                if 'kW' in variklis:
                     galia = int(variklis.split()[-2])
                else: galia = 'Nenurodyta'
                rida = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[7].text.strip().replace(' ','')[:-2]
                miestas = skelbimas.find('div', {'class':'announcement-parameters-block'}).find('div', {'class':'announcement-parameters'}).contents[9].text.strip()
            else:
                kuras = 'Patikrinti'
                pavara = 'Patikrinti'
                variklis = 'Patikrinti'
                turis = 'Patikrinti'
                galia = 'Patikrinti'
                rida = 'Patikrinti'
                miestas = 'Patikrinti'

            kuro_rusis.append(kuras)
            pavaros.append(pavara)
            turiai.append(turis)
            galios.append(galia)
            ridos.append(rida)
            miestai.append(miestas)
                                        
driver.close()

autoplius_data = pd.DataFrame()
autoplius_data['gamintojai'] = gamintojai
autoplius_data['modeliai'] = modeliai
autoplius_data['metai'] = metai
autoplius_data['kebulo_tipas'] = kebulo_tipas
autoplius_data['kuro_rusis'] = kuro_rusis
autoplius_data['pavaros'] = pavaros
autoplius_data['turiai'] = turiai
autoplius_data['galios'] = galios
autoplius_data['ridos'] = ridos
autoplius_data['miestai'] = miestai
autoplius_data['kainos'] = kainos

autoplius_data.to_csv('autoplius_data.csv', sep=';')